In [ ]:
import ipywidgets as wdg
import serial
from serial.tools import list_ports
import time
import threading

import numpy as np
import bqplot as bq

In [ ]:
RESOLUTION = 12

In [ ]:
port_list = [port.device for port in list_ports.comports()]
port_list.sort()

In [ ]:
tabs = []


port = wdg.Dropdown(options=port_list, value=port_list[0], description='Port:')
connect = wdg.Button(description='Connect')
disconnect = wdg.Button(description='Disconnect', disabled=True)
tabs.append(('Connection', wdg.VBox([port, connect, disconnect])))

request = wdg.Button(description='Request', disabled=True)
request_interval = wdg.FloatLogSlider(value=0.1, min=-3, max=2, step=0.1, description='Interval (s):')
request_continuous = wdg.Checkbox(value=False, description='Continuous')
tabs.append(('Request', wdg.VBox([request, request_interval, request_continuous])))

response = wdg.Label(value='')
response_display = wdg.IntProgress(value=0, min=0, max=2**RESOLUTION-1, description='Value:')


x_data = np.array([])
y_data = np.array([])

x_sc = bq.LinearScale()
y_sc = bq.LinearScale(min=0, max=2**RESOLUTION-1)

line = bq.Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc})

ax_x = bq.Axis(scale=x_sc, label='Time')
ax_y = bq.Axis(scale=y_sc, orientation='vertical', label='Value')

fig = bq.Figure(marks=[line], axes=[ax_x, ax_y], title='ADC Value vs Time')

top_bar = wdg.Tab(children=[tab for title, tab in tabs], titles=[title for title, tab in tabs])
layout = wdg.VBox([top_bar, response, response_display, fig])

In [ ]:
ser = None

requesting = False
request_thread = None

times = []
voltages = []

In [ ]:
from tracemalloc import start


def connect_to_port(port_name):
    global ser
    
    try:
        ser.close()
    except:
        pass

    try:
        ser = serial.Serial(port_name, 115200)
    except serial.SerialException as e:
        print(e)
        return
    
    connect.disabled = True
    disconnect.disabled = False
    request.disabled = False

    connect.description = 'Connected'
    top_bar.set_title(0, f'Connection ({port_name})')

    print('Connected to', port_name)

def disconnect_from_port(b):
    global ser, requesting, request_thread

    if request_thread is not None:
        requesting = False
        request_thread.join()
        request_thread = None
        request.description = 'Request'
    
    try:
        ser.close()
    except:
        pass
    
    connect.disabled = False
    disconnect.disabled = True
    request.disabled = True

    connect.description = 'Connect'
    top_bar.set_title(0, 'Connection')

    print('Disconnected from', ser.port)

def request_thread_entry():
    global ser, requesting, x_data, y_data

    last_time = time.time()
    start_time = last_time

    while requesting:
        loop_start_time = time.time()
        ser.write(b's')
        value = ser.readline().decode('utf-8')

        value = value[12-RESOLUTION:]

        value_dec = int(value, 2)
        elapsed_time = time.time() - last_time
        last_time = time.time()
        response.value = f"{value} ({value_dec}) @ {1/elapsed_time:.2f} Hz"
        response_display.value = value_dec

        times.append(last_time - start_time)
        voltages.append(value_dec)

        # constrain times and voltages to last 10 seconds
        while times[-1] - times[0] > 10:
            times.pop(0)
            voltages.pop(0)

        line.x = times
        line.y = voltages

        if request_interval.value - (time.time() - loop_start_time) > 0:
            time.sleep(request_interval.value - (time.time() - loop_start_time))

def request_data(b):
    global ser, requesting, request_thread

    if request_thread is not None:
        requesting = False
        request_thread.join()
        request_thread = None
        request.description = 'Request'
        return

    if not request_continuous.value:
        ser.write(b's')
        value = ser.readline().decode('utf-8')

        # discard first 4 characters (temporarily, for 8 bit version)
        value = value[12-RESOLUTION:]

        value_dec = int(value, 2)
        # value is a binary string, print the binary string and the decimal value
        response.value = f"{value} ({value_dec})"
        response_display.value = value_dec

    else:
        requesting = True
        request.description = 'Stop'

        request_thread = threading.Thread(target=request_thread_entry)
        request_thread.start()

In [ ]:
connect.on_click(lambda b: connect_to_port(port.value))
disconnect.on_click(disconnect_from_port)
request.on_click(request_data)

In [ ]:
display(layout)